In [ ]:
import numpy as np
import pandas as pd
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
# Data augmentation & transformation
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalise((0.5,0.5,0.5), (0.5,0.5,0.5))])

# Getting the data
trainset = datasets.MNIST(path, download=True, train=True, transform=transform)
#trainset = datasets.FashionMNIST(path, download=True, train=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)# Dataloader

testset = datasets.MNIST(path, download=True, train=False, transform=transform)
#testset = datasets.FashionMNIST(path, download=True, train=False, transform=transform)
testloader = DataLoader(testset, batch_size=32, shuffle=True)# Dataloader

In [ ]:
# Example 1
# Define network architecture
model = nn.Sequential(
    nn.Linear(784, 128), 
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    #nn.Dropout(p=0.2),
    nn.Linear(64, 10),
    nn.Softmax(dim=1)
)

# loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01) # Define our optimizer

# Data
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1) # Flatten

# Forward pass
logits = model.forward(images)
loss = criterion(logits, labels) # Calculate loss

loss.backward() # Calculate the Gradients for each layer

optimizer.step() # Taking 1 step and update the weights


# Calculating accuracy on test data
images, labels = next(iter(testloader))
images = images.view(images.shape[0], -1) # Flatten

# switch off gradients to save computation cost
with torch.no_grad():
    # model.eval() # if using dropout layers
    logits = model.forward(images)
    top_prob, top_class = logits.topk(1, dim=1) # Calculating the top probability and predicted class
    equals = top_class == labels.view(*top_class.shape) # if the class equals the labels
    torch.mean(equals.type(torch.FloatTensor))

In [ ]:
# Using a loop to train
epochs = 10
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1) # Flatten
        
        output = model.forward(images) # Forward pass
        loss = criterion(output, labels) # calculate loss
        running_loss += loss.item()
        
        loss.backward() # Calculate gradients
        
        optimizer.step() # Update weights
        optimizer.zero_grad() # zero out gradients
        
    print(f"Training loss: {running_loss/len(trainloader)}")s

In [ ]:
# Example 2
class Simple_Network(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        
        self.hidden1 = nn.Linear(input_size, 128)
        self.hidden2 = nn.Linear(128, 128)
        
        self.output = nn.Linear(128, 10)
        
    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = F.softmax(self.output(x), dim=1)
        
        return x